## SQL-like Window Functions in Pandas


> https://engineeringfordatascience.com/posts/sql_like_window_functions_in_pandas/

---

In [1]:
import datetime
import random

import ffn
import matplotlib.pyplot as plt

import pandas as pd

In [13]:
tickers = [
    "AAPL",  # apple
    "DIS",  # disney
    "NKE",  # nike
    "TSLA",  # tesla
]

# get stock price data
prices = ffn.get(tickers, start="2018-01-01")

# convert data into 'long' table format for purposes of this exercise
prices = prices.melt(ignore_index=False, var_name="ticker", value_name="closing_price")

# reset index to make 'Date' a column
prices = prices.reset_index()

# display 5 example rows in the dataset
prices.sample(5).sort_index()

,Date,ticker,closing_price
1506,2023-12-27,aapl,192.642548
2572,2021-10-20,dis,169.995972
2664,2022-03-03,dis,145.097122
5139,2019-03-06,tsla,18.416000
5826,2021-11-23,tsla,369.676666


### Example 1: Calculating the maximum stock price for each company in the time period#


In [14]:
prices['max_price'] = prices.groupby('ticker')['closing_price'].transform("max")

prices

,Date,ticker,closing_price,max_price
0,2018-01-02,aapl,40.615891,197.589523
1,2018-01-03,aapl,40.608810,197.589523
2,2018-01-04,aapl,40.797432,197.589523
3,2018-01-05,aapl,41.261932,197.589523
4,2018-01-08,aapl,41.108673,197.589523
...,...,...,...,...
6455,2024-05-28,tsla,176.750000,409.970001
6456,2024-05-29,tsla,176.190002,409.970001
6457,2024-05-30,tsla,178.789993,409.970001
6458,2024-05-31,tsla,178.080002,409.970001


### Example 2: 28 day closing price moving average for each company

In [16]:
prices

,Date,ticker,closing_price,max_price
0,2018-01-02,aapl,40.615891,197.589523
1,2018-01-03,aapl,40.608810,197.589523
2,2018-01-04,aapl,40.797432,197.589523
3,2018-01-05,aapl,41.261932,197.589523
4,2018-01-08,aapl,41.108673,197.589523
...,...,...,...,...
6455,2024-05-28,tsla,176.750000,409.970001
6456,2024-05-29,tsla,176.190002,409.970001
6457,2024-05-30,tsla,178.789993,409.970001
6458,2024-05-31,tsla,178.080002,409.970001


In [17]:
prices.dtypes

Date             datetime64[ns]
ticker                   object
closing_price           float64
max_price               float64
dtype: object

In [18]:
ex2_pandas = prices.copy()
ex2_pandas = ex2_pandas.sort_values(by="Date")

In [19]:
ex2_pandas = ex2_pandas.set_index("Date")

In [20]:
ex2_pandas

,ticker,closing_price,max_price
Date,,,
2018-01-02,aapl,40.615891,197.589523
2018-01-02,nke,59.444412,172.454758
2018-01-02,tsla,21.368668,409.970001
2018-01-02,dis,108.372864,201.254089
2018-01-03,aapl,40.608810,197.589523
...,...,...,...
2024-05-31,aapl,192.250000,197.589523
2024-06-03,dis,102.959999,201.254089
2024-06-03,aapl,193.610001,197.589523


In [25]:
ex2_pandas["ma_28_day"]  = ex2_pandas.sort_values("Date").groupby("ticker")["closing_price"].transform(lambda x: x.rolling(28, min_periods=1).mean())

In [26]:
ex2_pandas

,ticker,closing_price,max_price,ma_28_day
Date,,,,
2018-01-02,aapl,40.615891,197.589523,40.615891
2018-01-02,nke,59.444412,172.454758,59.444412
2018-01-02,tsla,21.368668,409.970001,21.368668
2018-01-02,dis,108.372864,201.254089,108.372864
2018-01-03,aapl,40.608810,197.589523,40.612350
...,...,...,...,...
2024-05-31,aapl,192.250000,197.589523,182.648107
2024-06-03,dis,102.959999,201.254089,183.610110
2024-06-03,aapl,193.610001,197.589523,92.744286


### Example 3: Get previous day’s closing share price for each ticker

In [27]:
ex3_pandas = prices.copy()
ex3_pandas = ex3_pandas.sort_values(by="Date")

In [31]:
ex3_pandas = ex3_pandas.sort_values(by=['ticker', 'Date'])

ex3_pandas

,Date,ticker,closing_price,max_price
0,2018-01-02,aapl,40.615891,197.589523
1,2018-01-03,aapl,40.608810,197.589523
2,2018-01-04,aapl,40.797432,197.589523
3,2018-01-05,aapl,41.261932,197.589523
4,2018-01-08,aapl,41.108673,197.589523
...,...,...,...,...
6455,2024-05-28,tsla,176.750000,409.970001
6456,2024-05-29,tsla,176.190002,409.970001
6457,2024-05-30,tsla,178.789993,409.970001
6458,2024-05-31,tsla,178.080002,409.970001


In [32]:
ex3_pandas.groupby('ticker')['closing_price'].transform(lambda x: x.shift(1))

0              NaN
1        40.615891
2        40.608810
3        40.797432
4        41.261932
           ...    
6455    179.240005
6456    176.750000
6457    176.190002
6458    178.789993
6459    178.080002
Name: closing_price, Length: 6460, dtype: float64

### Example 4: Daily Percentage Return#


In [33]:
ex4_pandas = prices.copy()

In [34]:
ex4_pandas = ex4_pandas.sort_values(by=['ticker', 'Date'])

ex4_pandas

,Date,ticker,closing_price,max_price
0,2018-01-02,aapl,40.615891,197.589523
1,2018-01-03,aapl,40.608810,197.589523
2,2018-01-04,aapl,40.797432,197.589523
3,2018-01-05,aapl,41.261932,197.589523
4,2018-01-08,aapl,41.108673,197.589523
...,...,...,...,...
6455,2024-05-28,tsla,176.750000,409.970001
6456,2024-05-29,tsla,176.190002,409.970001
6457,2024-05-30,tsla,178.789993,409.970001
6458,2024-05-31,tsla,178.080002,409.970001


In [35]:
ex4_pandas['last_day_price'] = ex4_pandas.groupby('ticker')['closing_price'].transform(lambda x: x.shift(1))

In [37]:
ex4_pandas['perc_return'] = (ex4_pandas['closing_price'] - ex4_pandas['last_day_price'])*100/ex4_pandas['last_day_price']

In [38]:
ex4_pandas

,Date,ticker,closing_price,max_price,last_day_price,perc_return
0,2018-01-02,aapl,40.615891,197.589523,NaN,NaN
1,2018-01-03,aapl,40.608810,197.589523,40.615891,-0.017432
2,2018-01-04,aapl,40.797432,197.589523,40.608810,0.464484
3,2018-01-05,aapl,41.261932,197.589523,40.797432,1.138553
4,2018-01-08,aapl,41.108673,197.589523,41.261932,-0.371430
...,...,...,...,...,...,...
6455,2024-05-28,tsla,176.750000,409.970001,179.240005,-1.389202
6456,2024-05-29,tsla,176.190002,409.970001,176.750000,-0.316830
6457,2024-05-30,tsla,178.789993,409.970001,176.190002,1.475674
6458,2024-05-31,tsla,178.080002,409.970001,178.789993,-0.397109


### Example 5: Missing Data Interpolation#

In [39]:
# copy orginal dataframe
ex5_pandas = prices.copy()

# remove 30% of data randomly
pct_missing = 0.3
num_missing = int(pct_missing * len(ex5_pandas))
indexes = random.sample(range(len(ex5_pandas)), k=num_missing)
mask = [i in indexes for i in range(len(ex5_pandas))]

# mask the dataframe with some random NaNs
ex5_pandas["closing_price"] = ex5_pandas["closing_price"].mask(mask)

In [40]:
# interpolate missing data paritioned by ticker
ex5_pandas["closing_price_interpolated"] = (
    ex5_pandas.sort_values("Date")
    .groupby("ticker")["closing_price"]
    .transform(lambda x: x.interpolate(method="ffill"))
)

ex5_pandas

,Date,ticker,closing_price,max_price,closing_price_interpolated
0,2018-01-02,aapl,40.615891,197.589523,40.615891
1,2018-01-03,aapl,40.608810,197.589523,40.608810
2,2018-01-04,aapl,40.797432,197.589523,40.797432
3,2018-01-05,aapl,41.261932,197.589523,41.261932
4,2018-01-08,aapl,41.108673,197.589523,41.108673
...,...,...,...,...,...
6455,2024-05-28,tsla,176.750000,409.970001,176.750000
6456,2024-05-29,tsla,NaN,409.970001,176.750000
6457,2024-05-30,tsla,178.789993,409.970001,178.789993
6458,2024-05-31,tsla,178.080002,409.970001,178.080002
